## T2 - A budget problem

PuLP is an LP modeler solving linear programming problems. You need to install PuLP before using. If you are using Anaconda/Miniconda, open the Conda Terminal and install PuLP:

    (base) C:\Users\YourName>conda activate env_tutorial
    (env_tutorial) C:\Users\YourName>conda install -c conda-forge pulp

In [1]:
import numpy as np
import pandas as pd
from pulp import *

### Solve for the budget problem

To solve for a LP problem, the basic recipe is
1. Create the model
2. Initiate decision variables
3. Add constraints
4. Add objective function
5. Specify solver
6. Check the result

In [2]:
# Create the model
model = LpProblem(name="budget-problem", sense=LpMinimize)

In [3]:
# Initialize the decision variables
x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)
x3 = LpVariable(name="x3", lowBound=0)
x4 = LpVariable(name="x4", lowBound=0)

print("Decision Variable: ")
print([x1,x2,x3,x4])

Decision Variable: 
[x1, x2, x3, x4]


In [4]:
# Add the constraints to the model
model += (400*x1 + 200*x2 + 150*x3 + 500*x4 >= 500, "calories")
model += (3*x1 + 2*x2 + 0*x3 + 0*x4 >= 6, "chocolate")
model += (2*x1 + 2*x2 + 4*x3 + 4*x4 >= 10, "sugar")
model += (2*x1 + 4*x2 + 1*x3 + 5*x4 >= 8, "fat")

In [5]:
# Add the objective function to the model
c1 = 5.5
c2 = 10.2
c3 = 5.3
c4 = 10.8
obj_func = c1*x1 + c2*x2 + c3*x3 + c4*x4
model += obj_func

In [6]:
# Check the model
model

budget-problem:
MINIMIZE
5.5*x1 + 10.2*x2 + 5.3*x3 + 10.8*x4 + 0.0
SUBJECT TO
calories: 400 x1 + 200 x2 + 150 x3 + 500 x4 >= 500

chocolate: 3 x1 + 2 x2 >= 6

sugar: 2 x1 + 2 x2 + 4 x3 + 4 x4 >= 10

fat: 2 x1 + 4 x2 + x3 + 5 x4 >= 8

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous

In [7]:
# Specify the solver to be GLPK (optional)
path_to = r'C:\Users\jzhangfv\Miniconda3\envs\env_tutorial\Library\bin\glpsol.exe'
solver = GLPK_CMD(path=path_to)

In [8]:
# Solve the problem
status = model.solve(solver)

# status meaning: {0: 'Not Solved', 1: 'Optimal', -1: 'Infeasible', -2: 'Unbounded', -3: 'Undefined'}
status

1

In [9]:
# Check results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
     print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 22.3875
x1: 2.0
x2: 0.0
x3: 0.875
x4: 0.625


### An alternative: specify parameters as matrices


In [10]:
# An alternative way to specify decision variables:
n_dv = 4
variable_names = [str(i) for i in range(1, n_dv+1)]
DV_variables = LpVariable.matrix("X", variable_names, lowBound=0)
print(DV_variables)

[X_1, X_2, X_3, X_4]


In [11]:
# RHS matrix
rhs_matrix = np.array([[400, 200, 150, 500],
                       [3, 2, 0, 0],
                       [2, 2, 4, 4],
                       [2, 4, 1, 5]])

# LHS Matrix
lhs_matrix = np.array([500, 6, 10, 8])

# Cost Matrix
cost_matrix = np.array([5.5, 10.2, 5.3, 10.8])

In [12]:
# Adding constraint via loop
model_alt = LpProblem(name="budget-problem-alt", sense=LpMinimize)
n_const = 4
for i in range(n_const):
    print(lpSum(rhs_matrix[i][j]*DV_variables[j] for j in range(n_dv)) >= lhs_matrix[i])
    model_alt += lpSum(rhs_matrix[i][j]*DV_variables[j] for j in range(n_dv)) >= lhs_matrix[i] , "Constraints " + str(i)

400*X_1 + 200*X_2 + 150*X_3 + 500*X_4 >= 500
3*X_1 + 2*X_2 >= 6
2*X_1 + 2*X_2 + 4*X_3 + 4*X_4 >= 10
2*X_1 + 4*X_2 + X_3 + 5*X_4 >= 8


In [13]:
# Add objective function
print(lpSum(DV_variables*cost_matrix))
model_alt += lpSum(DV_variables*cost_matrix)

5.5*X_1 + 10.2*X_2 + 5.3*X_3 + 10.8*X_4


In [14]:
# Check model formulation
model_alt

budget-problem-alt:
MINIMIZE
5.5*X_1 + 10.2*X_2 + 5.3*X_3 + 10.8*X_4 + 0.0
SUBJECT TO
Constraints_0: 400 X_1 + 200 X_2 + 150 X_3 + 500 X_4 >= 500

Constraints_1: 3 X_1 + 2 X_2 >= 6

Constraints_2: 2 X_1 + 2 X_2 + 4 X_3 + 4 X_4 >= 10

Constraints_3: 2 X_1 + 4 X_2 + X_3 + 5 X_4 >= 8

VARIABLES
X_1 Continuous
X_2 Continuous
X_3 Continuous
X_4 Continuous

In [15]:
# Solve the problem
model_alt.solve(solver)

# Check results
print(f"status: {model_alt.status}, {LpStatus[model_alt.status]}")
print(f"objective: {model_alt.objective.value()}")
for var in model_alt.variables():
     print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 22.3875
X_1: 2.0
X_2: 0.0
X_3: 0.875
X_4: 0.625


### What if $X_i$s are integer number?

In [16]:
# Specify variables' category as "Integer"
DV_variables = LpVariable.matrix("X", variable_names, cat = "Integer", lowBound=0)
print(DV_variables)

[X_1, X_2, X_3, X_4]


In [17]:
# Add constraints
model_int = LpProblem(name="budget-problem-int", sense=LpMinimize)

for i in range(n_const):
    print(lpSum(rhs_matrix[i][j]*DV_variables[j] for j in range(n_dv)) >= lhs_matrix[i])
    model_int += lpSum(rhs_matrix[i][j]*DV_variables[j] for j in range(n_dv)) >= lhs_matrix[i] , "Constraints " + str(i)

400*X_1 + 200*X_2 + 150*X_3 + 500*X_4 >= 500
3*X_1 + 2*X_2 >= 6
2*X_1 + 2*X_2 + 4*X_3 + 4*X_4 >= 10
2*X_1 + 4*X_2 + X_3 + 5*X_4 >= 8


In [18]:
# Add objective function
model_int += lpSum(DV_variables*cost_matrix)
model_int

budget-problem-int:
MINIMIZE
5.5*X_1 + 10.2*X_2 + 5.3*X_3 + 10.8*X_4 + 0.0
SUBJECT TO
Constraints_0: 400 X_1 + 200 X_2 + 150 X_3 + 500 X_4 >= 500

Constraints_1: 3 X_1 + 2 X_2 >= 6

Constraints_2: 2 X_1 + 2 X_2 + 4 X_3 + 4 X_4 >= 10

Constraints_3: 2 X_1 + 4 X_2 + X_3 + 5 X_4 >= 8

VARIABLES
0 <= X_1 Integer
0 <= X_2 Integer
0 <= X_3 Integer
0 <= X_4 Integer

In [19]:
# Solve the problem
model_int.solve(solver)

# Check results
print(f"status: {model_int.status}, {LpStatus[model_int.status]}")
print(f"objective: {model_int.objective.value()}")
for var in model_int.variables():
     print(f"{var.name}: {var.value()}")

status: 1, Optimal
objective: 26.5
X_1: 2
X_2: 1
X_3: 1
X_4: 0
